In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import pipeline
model_name = 'microsoft/phi-2'  # https://huggingface.co/models
device = 'cpu'

/home/heichtai/Projects/tutorial-finetune-LLM/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # Different quantizations are available, but require CUDA
    torch_dtype=torch.float32,
    # torch_dtype=torch.float16,
    # load_in_8bit=True,
    # load_in_4bit=True,
    trust_remote_code=True,
    revision="refs/pr/23",
)
model.eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.43it/s]


PhiForCausalLM(
  (transformer): PhiModel(
    (embd): Embedding(
      (wte): Embedding(51200, 2560)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (h): ModuleList(
      (0-31): 32 x ParallelBlock(
        (ln): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): Linear(in_features=2560, out_features=7680, bias=True)
          (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (inner_attn): SelfAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
          (inner_cross_attn): CrossAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
        )
        (mlp): MLP(
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
          (act): NewGELUActivation()
        )
      )
    )
  )
  (lm

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          use_fast=True,
                                          torch_dtype=torch.float32,
                                          )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
prompt = "Apple is red, brick is red, blood is red, and panda is"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

In [6]:
with torch.no_grad():
    output = model.generate(input_ids,
                            do_sample=True,
                            top_k=50,
                            top_p=0.95,
                            temperature=0.8,
                            pad_token_id=tokenizer.eos_token_id,
                            eos_token_id=tokenizer.eos_token_id,
                            max_new_tokens=8,  # For the sake of time. Feel free to change
                            )
print(tokenizer.decode(output[0], skip_special_tokens=True))

Apple is red, brick is red, blood is red, and panda is black. OUTPUT: Apple is red
